In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

print("--- MENCARI FILE & MEMBACA SECARA OTOMATIS ---")

# 1. Cari file berdasarkan kata kunci
files = os.listdir()
file_siswa = next((f for f in files if "siswa" in f.lower() and "sd" in f.lower() and ".xlsx" in f), None)
file_guru = next((f for f in files if "guru" in f.lower() and "sd" in f.lower() and ".xlsx" in f), None)
file_kelas = next((f for f in files if "kelas" in f.lower() and "sd" in f.lower() and ".xlsx" in f), None)

if None in [file_siswa, file_guru, file_kelas]:
    print("❌ ERROR: File Excel (.xlsx) belum ditemukan.")
    print("Pastikan Anda meng-upload file ASLI dari Kemdikbud (format .xlsx), bukan hasil convert.")
else:
    print(f"✅ FILE EXCEL DITEMUKAN:\n 1. {file_siswa}\n 2. {file_guru}\n 3. {file_kelas}")

    # 2. Baca File Excel (Gunakan read_excel, BUKAN read_csv)
    df_siswa = pd.read_excel(file_siswa, header=2)
    df_guru = pd.read_excel(file_guru, header=2)
    df_kelas = pd.read_excel(file_kelas, header=2)

    # 3. Bersihkan Nama Kolom (Hapus spasi yang tidak perlu)
    df_siswa.columns = df_siswa.columns.str.strip()
    df_guru.columns = df_guru.columns.str.strip()
    df_kelas.columns = df_kelas.columns.str.strip()

    # 4. Gabungkan Data (Merge)
    df_gabung = pd.merge(df_siswa, df_guru, on=['Kecamatan', 'Status'], how='inner', suffixes=('_Siswa', '_Guru'))
    df_final = pd.merge(df_gabung, df_kelas, on=['Kecamatan', 'Status'], how='inner')

    # 5. Pilih & Rapikan Kolom Penting
    df_clean = pd.DataFrame()
    df_clean['Lokasi'] = df_final['Kecamatan'] + " (" + df_final['Status'] + ")"

    # Biasanya: Jumlah_Siswa (dari suffix tadi), Jumlah_Guru, dan Jumlah (dari file kelas)
    df_clean['Total_Siswa'] = df_final['Jumlah_Siswa']
    df_clean['Total_Guru'] = df_final['Jumlah_Guru']

    # Kolom kelas biasanya bernama 'Jumlah' (karena dia file terakhir yang di-merge)
    if 'Jumlah' in df_final.columns:
        df_clean['Total_Kelas'] = df_final['Jumlah']
    else:
        # Jika tidak ada, cari kolom lain yg mengandung 'Jumlah'
        col_kelas = [c for c in df_final.columns if 'Jumlah' in c and c not in ['Jumlah_Siswa', 'Jumlah_Guru']][-1]
        df_clean['Total_Kelas'] = df_final[col_kelas]

    # Hitung Kelas Rusak
    df_clean['Kelas_Rusak'] = df_final['Rusak Sedang'] + df_final['Rusak Berat']

    # 6. Buat Target Label Otomatis (Sistem Deteksi Dini)
    def cek_status(row):
        if row['Total_Guru'] == 0: return 'Data Error'

        rasio_guru = row['Total_Siswa'] / row['Total_Guru']
        persen_rusak = (row['Kelas_Rusak'] / row['Total_Kelas']) * 100 if row['Total_Kelas'] > 0 else 0

        # Logika: Jika 1 Guru pegang > 28 murid ATAU Kerusakan > 30% -> WASPADA
        if rasio_guru > 28 or persen_rusak > 30:
            return 'WASPADA (Berisiko)'
        return 'AMAN'

    df_clean['Prediksi_Status'] = df_clean.apply(cek_status, axis=1)

    print("\n--- DATASET FINAL SIAP SCREENSHOT ---")
    display(df_clean)

--- MENCARI FILE & MEMBACA SECARA OTOMATIS ---
✅ FILE EXCEL DITEMUKAN:
 1. jumlah-siswa-menurut-kelompok-umur-tiap-provinsi-kab-gowa-sd-2022.xlsx
 2. jumlah-kepala-sekolah-dan-guru-menurut-golongan-tiap-provinsi-kab-gowa-sd-2021.xlsx
 3. jumlah-ruang-kelas-menurut-kondisi-tiap-propinsi-kab-gowa-sd-2024.xlsx

--- DATASET FINAL SIAP SCREENSHOT ---


,Lokasi,Total_Siswa,Total_Guru,Total_Kelas,Kelas_Rusak,Prediksi_Status
0,Kec. Bontonompo (Negeri),4157.0,283.0,207.0,97.0,WASPADA (Berisiko)
1,Kec. Bajeng (Negeri),5897.0,373.0,240.0,111.0,WASPADA (Berisiko)
2,Kec. Pallangga (Negeri),12095.0,534.0,296.0,99.0,WASPADA (Berisiko)
3,Kec. Somba Opu (Negeri),13180.0,635.0,280.0,111.0,WASPADA (Berisiko)
4,Kec. Bontomarannu (Negeri),4801.0,237.0,138.0,32.0,AMAN
5,Kec. Parang Loe (Negeri),1874.0,159.0,124.0,48.0,WASPADA (Berisiko)
6,Kec. Tinggi Moncong (Negeri),2484.0,182.0,149.0,64.0,WASPADA (Berisiko)
7,Kec. Bungaya (Negeri),1336.0,159.0,112.0,55.0,WASPADA (Berisiko)
8,Kec. Tompobulu (Negeri),2601.0,216.0,140.0,37.0,AMAN
9,Kec. Barombong (Negeri),4074.0,179.0,114.0,58.0,WASPADA (Berisiko)


In [ ]:
from google.colab import files

# 1. Simpan dulu DataFrame (df_clean) ke file CSV
# (Pastikan Anda sudah menjalankan kode penggabungan data sebelumnya sehingga df_clean sudah ada)
try:
    df_clean.to_csv("dataset_final_sekolah.csv", index=False)
    print("✅ Berhasil menyimpan file: dataset_final_sekolah.csv")

    # 2. Download file tersebut ke komputer
    files.download('dataset_final_sekolah.csv')
    print("⏳ Sedang mendownload...")

except NameError:
    print("❌ ERROR: Tabel 'df_clean' belum ada.")
    print("👉 SOLUSI: Jalankan dulu kode penggabungan data (Merge) di atas sebelum download.")
except Exception as e:
    print(f"❌ Terjadi error lain: {e}")

✅ Berhasil menyimpan file: dataset_final_sekolah.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⏳ Sedang mendownload...
